In [6]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [7]:
import ipywidgets as widgets
from IPython.display import display
import json
from beta import Artifact

def on_file_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        file_content = change['new']
        if not file_content:
            print("No file selected")
            return
        
        print("File content type:", type(file_content))
        print("File content:", file_content)
        
        # Handle the tuple structure
        if isinstance(file_content, tuple) and len(file_content) > 0:
            file_data = file_content[0]
            file_name = file_data['name']
            
            print(f"Selected file: {file_name}")
            print("File data type:", type(file_data))
            
            try:
                # Create the Artifact
                artifact = Artifact(files=[file_name])
                print("Artifact created successfully")
                print("Artifact DataFrame:")
                artifact.df.collect().show()
            except Exception as e:
                print(f"Error creating Artifact: {str(e)}")
        else:
            print("Unexpected file content format")

# Create a file upload widget
file_upload = widgets.FileUpload(
    accept='',  # Accept all file types
    multiple=False  # Allow only single file selection
)

# Display the file upload widget
display(file_upload)

# Attach the callback to the file upload widget
file_upload.observe(on_file_selected, names='value')

FileUpload(value=(), description='Upload')

In [8]:
from beta.agents import Agent
from beta.models.obj.model_object import VLLM
from vllm import LLM
from beta.tools.obj import CalculatorTool, DataFrameTool
from beta.models.serve.engines import VLLMEngine
from beta.models.serve.engines import OpenAIEngineConfig
from mlflow.client import MlflowClient
from daft import DataFrame
from beta.models.serve.engines.openai_engine import OpenAIEngine

In [9]:
mlflow_client = MlflowClient()
config = OpenAIEngineConfig(data={
    "api_key": "sk-svcacct-YRsA6cTdakp9lkCfpMh_hd8SmiTetZfn-M_SkJjrQbCQz9RquIajbiBdujKHertqBzZT3BlbkFJp5QdXnRMPYSt4or_JDvYz3IGIpZaNBOz-IlqKT4M7Jt5EQ4fAbCZdqhKH_rMaf6xwAA",
    "temperature": 0.5,
    "max_tokens": 4096,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
})
engine = OpenAIEngine(model_name="gpt-4o", mlflow_client=mlflow_client, config=config)
response = await engine.generate_structured("What is the capital of the moon?", structure="{moon: str, capital: str}")
print(response)


INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "moon": "Moon",
  "capital": "The Moon does not have a capital."
}


In [10]:
mlflow_client = MlflowClient()
config = OpenAIEngineConfig(data={"temperature": 0.5, "max_tokens": 4096, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0})
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct")
vllm = VLLM(model=LLM)
engine = VLLMEngine(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", mlflow_client=mlflow_client, config=config, api_key="EMPTY", base_url="http://localhost:8000")

paligemma = vllm(
    model_name="google/paligemma-3b-mix-448",
    engine=engine,
)

INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0


WARNING 09-20 04:32:27 arg_utils.py:839] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 09-20 04:32:27 config.py:911] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 09-20 04:32:27 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_p

NotImplementedError: 

In [13]:
from beta.prompts import prompt

@prompt
def my_prompt(question):
    """{{ question }}"""

@prompt
def cot(my_prompt, max_steps):
    """
    System: 
        Using Chain of Thought:
        max_steps = {{max_steps}}

        for step in max_steps:
            Reason about the request and append, "Therefore"
        
        Make a concluding response 

    User:
    {{ my_prompt }}
    """

In [46]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date
from ray import serve
@serve.deployment(name="Test0")
class QuarterlyEarnings(BaseModel):
    company_name: str = Field(..., description="Name of the company")
    fiscal_quarter: int = Field(..., ge=1, le=4, description="Fiscal quarter (1-4)")
    fiscal_year: int = Field(..., description="Fiscal year")
    revenue: float = Field(..., description="Total revenue for the quarter")
    net_income: float = Field(..., description="Net income for the quarter")
    earnings_per_share: float = Field(..., description="Earnings per share")
    report_date: date = Field(..., description="Date of the earnings report")
    analyst_expectations_met: Optional[bool] = Field(None, description="Whether analyst expectations were met")
    key_highlights: Optional[List[str]] = Field(None, description="Key highlights from the earnings report")

    def __call__(self, *args, **kwargs):
        return "QuarterlyEarnings deployment is working"


In [49]:
import ray
from ray import serve
import asyncio

ray.init(ignore_reinit_error=True, logging_level="DEBUG")
app = QuarterlyEarnings.bind()

handle = serve.run(app, name="Test0")

ray.get(handle.remote())

# Add debug print statements
print("Ray initialized and Serve started")

from ray.serve.handle import DeploymentHandle
import ray.serve as serve
from beta.agents.agent import Metadata
from beta.inference.inference_engine import RayInferenceEngine



q1_agent = Agent(
    name="Q1 Agent",
    model=handle,
    embedding=handle,
    stt=handle,
    tools=[CalculatorTool, DataFrameTool],
    metadata=Metadata(name="Q1 Agent", description="Agent for Q1 Agent"),
    inference_adapter=RayInferenceEngine(handle),
    is_async=True
)

async def run_agent():
    try:
        print("Preparing to call agent")
        response = await q1_agent(
            prompt=my_prompt("What were the key highlights of the Q4 2024 earnings?"),
        )
        print("Agent response received:")
        print(response)
    except Exception as e:
        print(f"Error occurred while calling agent: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the async function
asyncio.get_event_loop().run_until_complete(run_agent())

2024-09-20 05:06:47,641	INFO worker.py:1619 -- Calling ray.init() again after it has already been called.
INFO 2024-09-20 05:06:47,645 serve 1767713 api.py:259 - Connecting to existing Serve app in namespace "serve". New http options will not be applied.


RuntimeError: Deploying application Test0 failed: The deployments ['Test0'] are UNHEALTHY.

In [17]:
print("Available deployments:")
print(serve.list_deployments())

if "Test0" not in serve.list_deployments():
    print("Test0 deployment not found. Creating a dummy deployment for testing.")
    
    @serve.deployment(name="Test0")
    class DummyModel:
        async def generate(self, prompt=None, messages=None):
            return "This is a dummy response for testing purposes."
    
    DummyModel.deploy()

Available deployments:


AttributeError: module 'ray.serve' has no attribute 'list_deployments'

In [ ]:
from beta.models.obj import llm
from beta.models.serve.engines import LitServeEngine, OpenAIEngine, OpenRouterEngine, VLLMEngine


completion_settings = llm.settings(
    temperature=0.5,
    max_tokens=4096,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# Defaults to OpenRouter
# Expects OPENROUTER_API_KEY to be defined in .env
gemini_free = llm(
    model_name="google/gemini-pro-1.5-flash",
    settings=completion_settings,
)

# If OPENAI_API_KEY, uses { OpenAi | AsyncOpenAI } clients instead of OpenRouter. 
gpt4o = llm(
    model_name="openai/o1-mini",
    settings=completion_settings,
)

# If ANTHROPIC_API_KEY, uses {Anthropic | AsyncAnthopic }
sonnet = llm(
    model_name="anthopic/claude-3.5-sonnet",
    settings = completion_settings,
)


# Specifying the Engine with OS Model
paligemma = llm(
    model_name="google/paligemma-3b-mix-448",
    engine=LitServeEngine(), 
    completion_settings=completion_settings
)



SyntaxError: invalid syntax (model_object.py, line 15)

In [ ]:
from beta.tools import CalculatorTool, DataframeTool

In [ ]:

from beta.agents import BaseAgent, SpeechAgent

artifact_df = daft.DataFrame()
node_df = daft.DataFrame()
document_df = daft.DataFrame()






earnings = Data.Projects.get("Earnings Q4 2024")
knowledge_graph = my_project.Graphs.get("latest")


q1_agent = Agent(
    llm=paligemma,
    tools=[CalculatorTool, DataframeTool],
    context=[earnings],
    memory={'short':[knowl]},
    planning=
    rethinking=
)


@prompt
def my_prompt(question):
    """{{ question}}"""

@prompt
def cot(my_prompt,max_steps):
    """
    System: 
        Using Chain of Thought:
        max_steps = {{max_steps}}

        for step in max_steps:
            Reason about the request and append, "Therefore"
        
        Make a concluding response 

    User:
    {{ my_prompt }}
    """

from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date

class QuarterlyEarnings(BaseModel):
    """
    Represents quarterly earnings data for a company.
    """
    company_name: str = Field(..., description="Name of the company")
    fiscal_quarter: int = Field(..., ge=1, le=4, description="Fiscal quarter (1-4)")
    fiscal_year: int = Field(..., description="Fiscal year")
    revenue: float = Field(..., description="Total revenue for the quarter")
    net_income: float = Field(..., description="Net income for the quarter")
    earnings_per_share: float = Field(..., description="Earnings per share")
    report_date: date = Field(..., description="Date of the earnings report")
    analyst_expectations_met: Optional[bool] = Field(None, description="Whether analyst expectations were met")
    key_highlights: Optional[List[str]] = Field(None, description="Key highlights from the earnings report")


response = await my_agent(
    prompt=
    response_model=[Address], 
        reponse_model_strict = False
)